### AIMS RW Python Standard Test Friday 24th November 2023

The accuracy of the computed solution $\hat{x}$ of the system of linear equations $Ax=b$ can sometimes be improved by a process called iterative refinement:

1. Solve $Ax = b$ with GE to get $\hat{x}$ the estimate for $x$. You will find helpful functions in Assignment 2 which has answers in the Exercises folder.
2. Compute the residual $r = b - A\hat{x}$
3. Solve $Ad = r$ to get $d$
4. Update $\hat{x}$ by adding $d$ to $\hat{x}$.     ( $\hat{x} \leftarrow \hat{x} + d$.)

Steps 2--4 are repeated until there is convergence to a more accurate solution. At each iteration, since the matrices $L$, $U$ have already been computed when solving $Ax=b$, they can be used to solve $Ad=r$ in two stages using FS and BS at a cost of $O(n^2)$.

Write a function `GE_ref`, based on your function `GE`, to compute a solution to $Ax=b$ using GE with iterative refinement. Investigate the performance of `GE_ref` compared to that of `GE` and `GEPP`
You may wish to use test cases of  the  different values of $n$ and random matrices as in Assignment 2 and/or test methods from the Jupyter notebook in Week 2 called  _QR Methods Demo_ which investigates the effect of different condition numbers $\kappa_2(A)$. You should look at times taken, errors and residuals

Your results should be well presented (with some graphs if possible in the time that you have) and a **comment must be made** on your observations. You decide on what investigations you can do in the time available. If you are a faster programmer you can do a greater variety of tests. This is an open-ended task just like a research investigation would be. We look forward to what you will find and tell us.

### Mark scheme
10 marks: Exceptional investigation, all working with many tests & graphs and good explanations/comments <br>
6+ GE_ref completed with some tests and a few graphs & good comment<br>
5 GE_ref completed with a few tests, no graphs + comment <br>
4 GE_ref not fully working but some results from GE & GEPP + comment <br>
0+ no attempt at GE_ref but some results from GE&GEPP and comment




In [18]:
import numpy as np
import time
import matplotlib.pyplot as plt

####
# def FS here

def FS(L, b):
    # Find dimension of L
    n = L.shape[0]
    
    # Initialise y to zero vector of correct length
    y = np.zeros(n)
    
    # loop over elements of y, starting from first one
    for j in range(n):
        y[j] = (b[j] - L[j, :j] @ y[:j]) / L[j, j] # compute y_j
        
    return y

## def BS here

def BS(U, y):
    # Find dimension of U
    n = U.shape[0]
    
    # Initialise x to zero vector of correct length
    x = np.zeros(n)
    
    # loop over elements of x, starting from last one
    for j in range(n-1, -1, -1):
        x[j] = (y[j] - U[j, j+1:] @ x[j+1:]) / U[j, j] # compute x_j
    
    return x


##def LU(A) here
def LU(A):
    # Find dimension of A
    n = A.shape[0]
    
    # Initialise L=I, U=A
    L = np.eye(n) 
    U = np.copy(A)

    for k in range(n - 1): # loop over columns 1 to n-1
        for j in range(k + 1, n): # loop over rows k+1 to n
            L[j, k] = U[j, k] / U[k ,k] # compute the multiplier l_jk
            U[j, k:] = U[j, k:] - L[j, k] * U[k, k:] # subtract a multiple of row k from row j
    
    return L, U
        
    
# def LUPP here

def LUPP(A):
    # Find dimension of A
    n = A.shape[0]
    
    # Initialise matrices L=I, U=A, P=I
    L = np.eye(n)
    U = np.copy(A)
    P = np.eye(n)
    
    for k in range(n - 1): # Loop over columns 0 to n-2
        
        # Find pivot i
        i = np.argmax(np.abs(U[k:, k])) # Find position of maximum entry in required range
        i += k # Shift index to start counting from row 0
        
        # Swap rows correspondingly
        if i != k:
            # swap rows in U, columns k to n
            U[[k, i], k:] = U[[i, k], k:]
            
            # swap rows in L, columns 1 to k-1
            L[[k, i], :k] = L[[i, k], :k]
            
            # swap rows in P
            P[[k, i], :] = P[[i, k], :]
        
        # Loop over rows and make entries below
        # diagonal in column k equal to 0
        for j in range(k + 1, n): # loop over rows k+1 to n
            L[j, k] = U[j, k] / U[k, k] # compute the multiplier l_jk
            U[j, k:] = U[j, k:] - L[j, k] * U[k, k:] # subtract a multiple of row k from row j
        
    return L, U, P


# def GE here

def GE(A, b):
    L, U = LU(A) # find LU factorisation of A
    y = FS(L, b) # solve Ly=b using forward substitution
    x = BS(U, y) # solve Ux=y using back substitution
    return x


# def GEPP here
    
def GEPP(A, b):
    L, U, P = LUPP(A) # find LU factorisation with permuation of A
    y = FS(L, P @ b) # solve Ly=Pb using forward substitution
    x = BS(U, y) # solve Ux=y using back substitution
    return x
   
    x = GE(A,b)    

    r = b - A @ x

    d = np.linalg.inv(A) @ r

    x = x + d


# add code here

In [11]:
# define your new function GE_ref here


def GE_ref(A, b):
    x = GE(A,b)
    while np.linalg.norm(A @ x - b, np.inf) >= 10**(-3):
        r = b - A @ x
        A = LU(A,b)[0] @ LU(A,b)[1]
        d = np.linalg.inv(A) @ r
        x = x + d
        
   
    return x

In [15]:
# Testing with different condition numbers

#Use the function randsvd QR Methods Demo Jupyter notebook if you wish


def randsvd(m, n, kappa):
    s = np.zeros(n)
    S = np.zeros((m,n))
    for i in range(n):
        beta = kappa**(1/(n-1))
        s[i] = beta**(-i)
    S[:n,:n] = np.diag(s)

    def haar(n):
        A = np.random.randn(n, n)
        Q, R = np.linalg.qr(A)

        for i in range(n):
            if R[i, i] < 0:
                Q[:, i] *= -1
        return Q

    A = haar(m) @ S @ haar(n).T
    
    
    
    return A

# Testing randsvd
m=10
n=5
kappa=1000
randsvd(m, n, kappa)


# this function tests the different GE methods on A in the func_list
def test_GE(A, func_list): 
    # Get xsol and b
    n = A.shape[0]
    xsol = np.ones(n) # xsol is a column of ones
    b = A @ xsol      # get the true solution b for xsol
    
    # List of residuals, times, and errors
    t = []
    r = []
    e = []
    
    # Solve system using GE, GEPP and GE_ref
    for fun in func_list:
        t0 = time.time()
        x = fun(A, b)
        t.append(time.time() - t0)
        r.append(np.linalg.norm(A @ x - b, np.inf))
        e.append(np.linalg.norm(xsol - x, np.inf))
    
    return t, r, e
A = np.array([[2, 1, 1], [4, 3, 3], [8, 7, 9]], dtype=float)
b = np.array([4, 10, 24], dtype=float)
func_list=[GE(A,b),GEPP(A,b),GE_ref(A,b)]

In [16]:
for k in range(4, 5): ## Change the range for your tests
    kappa = 10 ** k
    n = 400
    A = randsvd(n, n, kappa)
    
    func_list = [GE, GEPP, GE_ref]
    t, r, e = test_GE(A, func_list)
    
    print('Condition number: kappa = {:.4g}\n\n'
          'Residual:\n'
          'GE: {:.5g}\nGEPP: {:.5g}\nGE_ref: {:.5g}\n\n'
          'Error:\n'
          'GE: {:.5g}\nGEPP: {:.5g}\nGE_ref: {:.5g}\n\n'
          'Time:\n'
          'GE: {:.5g} s\nGEPP: {:.5g} s\nGE_ref: {:.5g} s\n\n'.format(kappa,
                                                                   r[0], r[1], r[2],
                                                                   e[0], e[1], e[2],
                                                                   t[0], t[1], t[2]))

Condition number: kappa = 1e+04

Residual:
GE: 1.1328e-13
GEPP: 1.7208e-15
GE_ref: 1.1328e-13

Error:
GE: 1.1897e-10
GEPP: 1.8268e-12
GE_ref: 1.1897e-10

Time:
GE: 0.27532 s
GEPP: 0.26218 s
GE_ref: 0.24865 s




In [ ]:
## add other tests using different types of  matrices and displaying
#results clearly, possibly with some graphs

Make a comment here about what you see in your results, comparing the different methods.

Comment:





The time in all the three methods is close to the same value, which means that as n tends to infinity, the cost is the same

The error of GEPP is the smallest which means that GEPP is the best and it is the same as regarding the three residuals.